<a href="https://colab.research.google.com/github/IgorWounds/Bayesian-Neural-Network-Algotrading101/blob/main/Bayesian_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install tensorflow-probability
!pip install tensorflow-datasets

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [3]:
import logging

logging.basicConfig(
    format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO
)

# Load data and get it ready

In [4]:
def train_test_split(train_size, batch_size=1):

    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=500)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test = dataset.skip(train_size).batch(batch_size)

    return train, test

In [5]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]

In [6]:
def create_inputs():
    inputs = {}
    for input in FEATURE_NAMES:
        inputs[input] = layers.Input(
            name=input, shape=(1,), dtype=tf.float32
        )

    return inputs

In [7]:
create_inputs()

{'alcohol': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'alcohol')>,
 'chlorides': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'chlorides')>,
 'citric acid': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'citric acid')>,
 'density': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'density')>,
 'fixed acidity': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fixed acidity')>,
 'free sulfur dioxide': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'free sulfur dioxide')>,
 'pH': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'pH')>,
 'residual sugar': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'residual sugar')>,
 'sulphates': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'sulphates')>,
 'total sulfur dioxide': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'total sulfur dioxide')>,
 'volatile acidity': <KerasTensor: shape=(None, 

In [8]:
train, test = train_test_split(4163, 256)

[2022-04-29 20:08:43,657] (INFO): No config specified, defaulting to first: wine_quality/white
[2022-04-29 20:08:43,660] (INFO): Load dataset info from /root/tensorflow_datasets/wine_quality/white/1.0.0
[2022-04-29 20:08:43,664] (INFO): Reusing dataset wine_quality (/root/tensorflow_datasets/wine_quality/white/1.0.0)
[2022-04-29 20:08:43,666] (INFO): Constructing tf.data.Dataset for split train, from /root/tensorflow_datasets/wine_quality/white/1.0.0


# Create the BNN

## Create the Prior

In [9]:
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior

## Create the Posterior


In [10]:
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior

## Create custom loss function

In [11]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)

## Create BNN

In [12]:
def bnn(train_size, hidden_units):
    inputs = create_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    print(features)
    features = layers.BatchNormalization()(features)
    print(features)
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="relu",
        )(features)

    # Generate a probabilistic result, and use the `Dense` layer
    # to produce the parameters of the said distribution.
    # Set units=2 to learn both the mean and the variance of the distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

## Compile and run

In [13]:
def run(model, loss, learning_rate, train, test):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    logging.info("Training BNN")
    model.fit(train, epochs=num_epochs, validation_data=test)

    logging.info("BNN Training Complete")

    _, rmse = model.evaluate(train, verbose=0)
    logging.info(f"Train RMSE: {round(rmse, 4)}")

    _, rmse = model.evaluate(test, verbose=0)
    logging.info(f"Test RMSE: {round(rmse, 4)}")

# Train and evaluate BNN

In [14]:
batch_size = 256
num_epochs = 650
learning_rate = 0.001
hidden = [8, 8]

bnn_model = bnn(4163, hidden)

run(
    bnn_model, negative_loglikelihood, learning_rate, train, test
)

KerasTensor(type_spec=TensorSpec(shape=(None, 11), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")
KerasTensor(type_spec=TensorSpec(shape=(None, 11), dtype=tf.float32, name=None), name='batch_normalization/batchnorm/add_1:0', description="created by layer 'batch_normalization'")


[2022-04-29 20:08:44,616] (INFO): Training BNN


Epoch 1/650


KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(bnn_model, to_file='model.png')